## PCLR

----

### 1. Import Modules

In [1]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


### 2. Load data and peak sheet

In [2]:
home = 'data/' 
file = 'MTBLS90.xlsx'  

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 968 TOTAL PEAKS: 189
Done!


### 3. Extract X & Y

In [3]:
peaklist = PeakTable['Name']
DataTable2 = DataTable[(DataTable['Class']==0) | (DataTable['Class']==1)]
Y = DataTable2['Class'].values 

### 4. Hyperparameters optimisation

In [ ]:
# Set seed
seed = 13

# Split DataTable2 and Y into train and test (with stratification
DataTrain, DataTest, Ytrain, Ytest = train_test_split(DataTable2, Y, test_size=0.30, stratify=Y, random_state=seed)

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTrain[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)                                      
                                         
# param_dict
#param_dict = {'n_components': [1,2,3,4,5,6]}                   
#param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12]} 
param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]} 

# Initalise
cv = cb.cross_val.kfold(model=cb.model.PCLR,                      
                                X=XTknn,                                 
                                Y=Ytrain,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=100)                                

# Run and plot
cv.run()  
cv.plot(metric='r2q2', ci=95)

Number of cores set to: 8
Running ...


100%|██████████| 3000/3000 [01:49<00:00, 27.48it/s]


Time taken: 1.83 minutes with 8 cores


### 6. Get Model and Evaluate

In [ ]:
# 6. Train, test and evaluate model

# Train model
model = cb.model.PCLR(n_components=22)
model.train(XTknn, Ytrain)

# Get X, Y
mu, sigma  = cb.utils.scale(XTlog, return_mu_sigma=True) 
peaklist = PeakTable.Name 
XV = DataTest[peaklist].values
XVlog = np.log(XV)
XVscale  = cb.utils.scale(XVlog, method='auto', mu=mu, sigma=sigma)
XVknn = cb.utils.knnimpute(XVscale, k=3)
YVpred = model.test(XVknn)

# Evaluate Ypred against Ytest
evals = [Ytest, YVpred] 
model.evaluate(evals, cutoffscore=0.5, plot_median=False) 

print("AUC Train: {}".format(model.table['auc'][0][0]))
print("AUC Test: {}".format(model.table['auc'][1][0]))

In [ ]:
# Get X
X = DataTable2[peaklist]                                    
Xlog = np.log(X)                                          
Xscale = cb.utils.scale(Xlog, method='auto')              
Xknn = cb.utils.knnimpute(Xscale, k=3)    

# Bootstrap evaluate
model.booteval(Xknn, Y, cutoffscore=0.5, bootnum=100) 

### 7. Save table

In [ ]:
home = 'tables/'
file = 'PCLR_MTBLS90.xlsx'

model.save_table(home + file)